In [1]:
# helpful discussions: 
# https://stackoverflow.com/questions/72640228/how-to-replace-a-string-in-a-file-without-changing-encoding

import glob
import datetime
import time

In [ ]:
# Renameing/deleting 

# ifg_data_folder = '/uufs/chpc.utah.edu/common/home/lin-group15/agm/em27/ua/inst_data/ifgs'
# date = '20230516'
# date_folder = os.path.join(ifg_data_folder,date)

# #Correct gap in ifg data
# good_data_ifgs = []
# no_data_ifgs = []

# for file in sorted(os.listdir(date_folder)):
#     fsize_kb = os.stat(os.path.join(date_folder,file)).st_size/1024
#     if fsize_kb>2:
#         good_data_ifgs.append(file)
#     else:
#         no_data_ifgs.append(file)

# for file in no_data_ifgs:
#     os.remove(os.path.join(date_folder,file))

# before_3600_ids = []
# after_3600_ids = []
# for file in sorted(os.listdir(date_folder)):
#     ifg_id =int(file.split('.')[-1])
#     if ifg_id < 3601:
#         before_3600_ids.append(ifg_id)
#     else:
#         after_3600_ids.append(ifg_id)
# last_before_3600_id = max(before_3600_ids)

# new_id = last_before_3600_id

# for old_id in after_3600_ids:
#     new_id = new_id+1
#     new_id_str = f'{new_id:04}'

#     old_fullname = os.path.join(date_folder,f'ua{date}.ifg.{old_id}')
#     new_fullname = os.path.join(date_folder,f'ua{date}.ifg.{new_id_str}')
#     os.rename(old_fullname,new_fullname)

# Part I: Look at just one file

In [6]:
# return the date and time strings, ensure that all printed output is what you expect
test_file = '/uufs/chpc.utah.edu/common/home/lin-group15/agm/em27/ua/inst_data/ifgs/20230516/ua20230516.ifg.4197'

print("File Path: " + test_file)
file = open(test_file, "rb")
s = file.read()
pos = s.find(b'DAT') # get the position of the string "DAT" in the ifg file
print("DAT position: " + str(pos)) #915428

# look for date string
# print(chr(s[pos]) + chr(s[pos+1]) + chr(s[pos+2]))
# print((s[pos:pos+3])) # prints "DAT"
date_start =  pos+8 # get index of where date string starts (8 characters from where the "DAT" string is found)
date_end = date_start + 10 # get index of where date string ends (10 characters from where the date string starts)
date_found = s[date_start:date_end] # get the complete date string using indices
print(date_found) # prints the date string in the file

# look for time string
pos = s.find(b'TIM') # get the position of the string "TIM" in the ifg file
time_start = pos+8 # get index of where time string starts (8 characters from where the "TIM" string is found)
time_end = time_start+20 # get index of where time string ends (20 characters from where the time string starts)
time_found = s[time_start:time_end] # get the complete time string using indices
print('TIM position: ' + str(pos))
print(time_found)

# # time_off = 42775 #in seconds
# # convert time_found to time object
# # add the timeoffset
# # convert back to string

file.close()

File Path: /uufs/chpc.utah.edu/common/home/lin-group15/agm/em27/ua/inst_data/ifgs/20230516/ua20230516.ifg.4197
DAT position: 915428
b'16/05/2023'
TIM position: 915448
b'22:36:48.154 (GMT+0)'


In [21]:
### Fix the time offset

## convert time string to time object 
tim_string = time_found.decode().split('.')[0]

## set date string
# dat_string = date_found.decode() # use date found in file
dat_string = "03/11/2022" # or if the date in the file is wrong, you can just indicate the correct date here

## convert date and time strings to datetime
test = datetime.datetime.strptime(dat_string + ' ' + tim_string, '%d/%m/%Y %H:%M:%S')
print("Original time: " + str(test))

## input time offset in seconds
time_off = datetime.timedelta(seconds = 61651)

## apply the time offset to get the correct time
new_time = test + time_off
print("Corrected time: " + str(new_time))

## create new byte object with the correct time string
new_time_str = str(new_time.time()) + '.704 (GMT+0)' # convert correct time to string, in the same format as in the ifg file
## note: I manually added the .704 to the end of the time so that the length of the time string would be the same length as 
##      what is in the ifg file. Without this it will not work, since it needs to replace text with a string with the same number
##      of characters. The times in the ifg files must be in some sort of decimal format, but I couldn't figure it out. I 
##      just accepted that my times may be off by ~1 second, but it's okay because I average it into 5 min anyway. 
new_time_byt = bytes(new_time_str, 'utf-8') # convert string to byte object
# print(new_time_byt)

Original time: 2022-11-03 22:56:29
Corrected time: 2022-11-04 16:04:00


In [62]:
## replace the time_found with the corrected time
file = open(test_file, "rb")
s = file.read() 
s2 = s.replace(time_found, new_time_byt) # create a copy of the original file, but replace the times with the correct time
s2 = s2.replace(date_found, new_dat_byt) # replace the date with the correct date
file.close()

In [64]:
## BE CAREFUL - THIS WILL OVERWRITE YOUR FILE!!!
## overwrite the file with the corrected time 
with open(test_file+'_test', 'wb') as output_file: # wb indicates we can write to the file
    output_file.write(s2)

# Part II: Loop through all files and update date/time strings

In [11]:
## same process as Part I, but now looping over all ifg files

time_off = datetime.timedelta(seconds = 61651) # input time offset in seconds
new_dat_byt = bytes("03/11/2022", 'utf-8') # DD/MM/YYYY

for f in ifgs_files:
#     print(f)
    file = open(f, "rb") # open the file
    s = file.read() # read the file
    
    ## find the date
    pos = s.find(b'DAT')
#     pos = 915428
    date_start = pos+8
    date_end = date_start + 10
    date_found = s[date_start:date_end]
#     print(date_found)
    
    # find the time
#     pos = s.find(b'TIM')
    pos = 915448 # some files were finding a fake "TIM" so I just set the position and it was the same for all files
    time_start = pos+8
    time_end = time_start+20
    time_found = s[time_start:time_end]
#     print(time_found)

    # convert time string to datetime object 
    tim_string = time_found.decode().split('.')[0]
    dat_string = date_found.decode()
    
    if len(dat_string) == 10:
        datetime_str = datetime.datetime.strptime(dat_string + ' ' + tim_string, '%d/%m/%Y %H:%M:%S')
#       print(datetime_str.time())
    
        # apply the time offset correction
        new_time = datetime_str + time_off
#         print(new_time.time())

        # convert time to the format in the binary file 
        new_time_str = str(new_time.time()) + '.704 (GMT+0)'
        new_time_byt = bytes(new_time_str, 'utf-8')
#         print(new_time_byt)

        # replace the time with the new time
        s2 = s.replace(time_found, new_time_byt)
        s2 = s2.replace(date_found, new_dat_byt)
        file.close()
        
        # write the whole file back
        with open(f, 'wb') as output_file:
            output_file.write(s2)
            print(f + " was overwritten")
        
        
    else:
        file.close()


E:/EM27/20221103\ua20221103.ifg.0001 was overwritten
E:/EM27/20221103\ua20221103.ifg.0002 was overwritten
E:/EM27/20221103\ua20221103.ifg.0003 was overwritten
E:/EM27/20221103\ua20221103.ifg.0004 was overwritten
E:/EM27/20221103\ua20221103.ifg.0005 was overwritten
E:/EM27/20221103\ua20221103.ifg.0006 was overwritten
E:/EM27/20221103\ua20221103.ifg.0007 was overwritten
E:/EM27/20221103\ua20221103.ifg.0008 was overwritten
E:/EM27/20221103\ua20221103.ifg.0009 was overwritten
E:/EM27/20221103\ua20221103.ifg.0010 was overwritten
E:/EM27/20221103\ua20221103.ifg.0011 was overwritten
E:/EM27/20221103\ua20221103.ifg.0012 was overwritten
E:/EM27/20221103\ua20221103.ifg.0013 was overwritten
E:/EM27/20221103\ua20221103.ifg.0014 was overwritten
E:/EM27/20221103\ua20221103.ifg.0015 was overwritten
E:/EM27/20221103\ua20221103.ifg.0016 was overwritten
E:/EM27/20221103\ua20221103.ifg.0017 was overwritten
E:/EM27/20221103\ua20221103.ifg.0018 was overwritten
E:/EM27/20221103\ua20221103.ifg.0019 was overw

In [12]:
# # check if it worked 
file = open(ifgs_files[10], 'rb')
s = file.read()

file.close()